In [1]:
import pandas as pd
import numpy as np
import time
import random
import lightgbm as lgb
import time

import pickle
import matplotlib.pyplot as plt

In [2]:
from process import map_pred

In [3]:
FOLDER = '../../data_kaggle/champs/'
OUTPUT = FOLDER + 'out/'
TEMP = OUTPUT + 'temp/'
MAP  = OUTPUT + 'map/'

In [4]:
# original input
df_train = pd.read_csv(FOLDER+'train.csv')
df_test = pd.read_csv(FOLDER+'test.csv')
mols_train = df_train['molecule_name'].unique()

In [5]:
DATA_DATE = '20190721'
MAP_DATE = '20190728'

In [6]:
# input files
map_train = MAP + MAP_DATE + '_' + 'map_train.pickle'
map_test = MAP + MAP_DATE + '_' + 'map_test.pickle'

first_train = OUTPUT + DATA_DATE + '_' + 'dist_ang_ori_bond_cos_train_{}.pickle'
first_train_all = OUTPUT + DATA_DATE + '_' + 'features_train.pickle'

first_test = OUTPUT + DATA_DATE + '_' + 'dist_ang_ori_bond_cos_test_{}.pickle'
first_test_all = OUTPUT + DATA_DATE + '_' + 'features_test.pickle'

In [7]:
df_map_train = pd.read_pickle(map_train)
df_map_train.head()

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,id,molecule_name
0,0,-1,-1,-1,-1,-1,-1,1,2,3,...,-1,-1,-1,-1,-1,-1,-1,-1,0,dsgdb9nsd_000001
1,4,-1,-1,-1,-1,-1,-1,6,5,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,4,dsgdb9nsd_000001
2,7,-1,-1,-1,-1,-1,-1,8,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,7,dsgdb9nsd_000001
3,9,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,9,dsgdb9nsd_000001
0,1,0,-1,-1,-1,-1,-1,2,3,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,1,dsgdb9nsd_000001


In [8]:
df_features_train = pd.DataFrame()
bond_types = ['1JHC', '1JHN', '2JHH', '2JHC', '2JHN', '3JHH', '3JHC', '3JHN']
for b in bond_types:
    df_features_train = pd.concat([df_features_train, pd.read_pickle(first_train.format(b))], axis=0)
df_features_train = df_features_train.sort_values('id').reset_index(drop=True)
df_features_train.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,dist_H_0_x,dist_H_1_x,dist_H_2_x,dist_H_3_x,...,orientation_O_2_y,dist_F_0_y,dist_F_1_y,angle_F_0_y,angle_F_1_y,orientation_F_0_y,orientation_F_1_y,cos_3j,cos_3j^2,dist_center
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,0.560815,0.560806,0.560803,0.0,...,1.0,0.0,0.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,0.560815,0.560806,0.560803,0.0,...,1.0,0.0,0.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,0.560815,0.560806,0.560803,0.0,...,1.0,0.0,0.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543,0.560815,0.560806,0.560803,0.0,...,1.0,0.0,0.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,0.560815,0.560806,0.560803,0.0,...,1.0,0.0,0.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN


In [9]:
df_features_train = df_features_train.fillna(0)
features_train = df_features_train[df_features_train.columns[6:]].values

In [10]:
with open(first_train_all, 'wb') as f:
    pickle.dump(features_train, f)

In [11]:
df_features_test = pd.DataFrame()
bond_types = ['1JHC', '1JHN', '2JHH', '2JHC', '2JHN', '3JHH', '3JHC', '3JHN']
for b in bond_types:
    df_features_test = pd.concat([df_features_test, pd.read_pickle(first_test.format(b))], axis=0)
df_features_test = df_features_test.sort_values('id').reset_index(drop=True)
df_features_test.head()

,id,molecule_name,atom_index_0,atom_index_1,type,dist_H_0_x,dist_H_1_x,dist_H_2_x,dist_H_3_x,dist_H_4_x,...,orientation_O_2_y,dist_F_0_y,dist_F_1_y,angle_F_0_y,angle_F_1_y,orientation_F_0_y,orientation_F_1_y,cos_3j,cos_3j^2,dist_center
0,4658147,dsgdb9nsd_000004,2,0,2JHC,0.300908,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN
1,4658148,dsgdb9nsd_000004,2,1,1JHC,0.300908,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN
2,4658149,dsgdb9nsd_000004,2,3,3JHH,0.300908,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.199079
3,4658150,dsgdb9nsd_000004,3,0,1JHC,0.300908,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN
4,4658151,dsgdb9nsd_000004,3,1,2JHC,0.300908,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN


In [12]:
df_features_test = df_features_test.fillna(0)
features_test = df_features_test[df_features_test.columns[5:]].values

In [13]:
with open(first_test_all, 'wb') as f:
    pickle.dump(features_test, f)